# Libraries

In [88]:
import networkx as nx
import pygame
import time

# Global Variables

In [89]:
# GLOBAL VARIABLES
WHITE = (255,255,255)
BLACK = (0,0,0)
GREY = (194,194,194)
BLUE = (25, 166, 194)
DARK_BLUE = (21, 134, 157)
RED = (255, 0, 0)
ORANGE = (234, 108, 94)
DARK_ORANGE = (236, 44, 22)
GREEN = (0, 255, 0)
LIME = (66, 213, 30)
DARK_LIME = (48, 157, 21)
BROWN = (205, 118, 66)
DARK_BROWN = (135, 76, 41)
OFFSET = 100


In [107]:
def round_better(number, significant_figures):
    from math import log10, floor
    
    if number == 0:
        return 0
    else:
        decimal_places = significant_figures - int(floor(log10(abs(number)))) - 1
        rounded_number = round(number, decimal_places)
        return rounded_number

# PyGame

In [90]:
class Button:
    def __init__(self, x, y, width, height, text, color, hover_color):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.color = color
        self.hover_color = hover_color
        self.font = pygame.font.Font(None, 36)
    
    def draw(self, screen):
        mouse_pos = pygame.mouse.get_pos()
        if self.rect.collidepoint(mouse_pos):
            pygame.draw.rect(screen, self.hover_color, self.rect)
        else:
            pygame.draw.rect(screen, self.color, self.rect)
        
        text_surf = self.font.render(self.text, True, BLACK)
        text_rect = text_surf.get_rect(center=self.rect.center)
        screen.blit(text_surf, text_rect)
    
    def is_clicked(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.rect.collidepoint(event.pos):
                return True
        return False

    def set_text(self, text):
        self.text = text

    def set_color(self, color):
        self.color = color

In [91]:
def draw_contour(screen, size, color):
    pygame.draw.rect(screen, color, pygame.Rect(400-(size/2) + OFFSET, 300-(size/2), size, size))
    
def draw_rounded_rect(screen, color, x, y, w, h, radius):
    pygame.draw.rect(screen, color, (x + radius, y, w - 2 * radius, h))
    pygame.draw.rect(screen, color, (x, y + radius, w, h - 2 * radius))
    pygame.draw.circle(screen, color, (x + radius, y + radius), radius)
    pygame.draw.circle(screen, color, (x + w - radius, y + radius), radius)
    pygame.draw.circle(screen, color, (x + radius, y + h - radius), radius)
    pygame.draw.circle(screen, color, (x + w - radius, y + h - radius), radius)
    
def draw_text(screen, text, x, y, font_size, color):
    font = pygame.font.Font(None, font_size)
    text = font.render(text, True, color)
    screen.blit(text, (x, y))
    
def draw_pieces(screen,positions):
    row = -1
    for i in range(9):
        if (i % 3 == 0): row += 1 
        if (positions[i] != ' '):
            draw_rounded_rect(screen, BLACK, (1.485*(i % 3) + 1.0017)*124 + OFFSET, (7.67*(row % 3) + 1.0017)*24, 184, 184, 10)
            draw_rounded_rect(screen, WHITE, (1.485*(i % 3) + 1.0017)*(124) + 2 + OFFSET, (7.67*(row % 3) + 1.0017)*(24) + 2, 180, 180, 10) 
            draw_text(screen, str(positions[i]), (1.485*(i % 3) + 1.53)*124 + OFFSET,(7.67*(row % 3) + 3)*24,150,BLACK)

            


# Search Algorithms

In [92]:
class AStar8Puzzle:
    @staticmethod
    def find_blank(array_matriz):
        index= array_matriz.index(' ')
        return index
    
    @staticmethod
    def get_score_index(index_state, index_solucion):
        row_1 = [0, 1, 2]
        row_2 = [3, 4, 5]
        row_3 = [6, 7, 8]

        col_1 = [0, 3, 6]
        col_2 = [1, 4, 7]
        col_3 = [2, 5, 8]

        state_x = 0
        state_y = 0
        sol_x = 0
        sol_y = 0

        if index_solucion in row_1:
            sol_y = 0
        if index_solucion in row_2:
            sol_y = 1
        if index_solucion in row_3:
            sol_y = 2
        if index_state in row_1:
            state_y = 0
        if index_state in row_2:
            state_y = 1
        if index_state in row_3:
            state_y = 2
        if index_solucion in col_1:
            sol_x = 0
        if index_solucion in col_2:
            sol_x = 1
        if index_solucion in col_3:
            sol_x = 2
        if index_state in col_1:
            state_x = 0
        if index_state in col_2:
            state_x = 1
        if index_state in col_3:
            state_x = 2

        return abs(sol_x - state_x) + abs(sol_y - state_y)
    
    @staticmethod
    def get_score(lista_1nodo, lista_goal):
        score=0
        for i in range(len(lista_1nodo)):
            if i==0:
                nodo_index=lista_1nodo.index(' ')
                goal_index=lista_goal.index(' ')
            else:
                nodo_index=lista_1nodo.index(i)
                goal_index=lista_goal.index(i)
            score= score + AStar8Puzzle.get_score_index(nodo_index,goal_index)
            
        return score
    
    @staticmethod
    def get_n_sucesores(array_matriz):
        acciones = []
        index = AStar8Puzzle.find_blank(array_matriz)
        if index  in [0,1,2,3,4,5]:
            acciones.append('down')
        if index in [0,3,6,1,4,7]:
            acciones.append('right')
        if index in [2,5,8,1,4,7]:
            acciones.append('left')
        if index in [6,7,8,3,4,5]:
            acciones.append('up')

        return acciones
    
    @staticmethod
    def generate_n_sucesores(lista_acciones, nodo_anterior):
        index=AStar8Puzzle.find_blank(nodo_anterior)
        lista_n_sucesores=[]
        while lista_acciones:
            aux=-1
            lista_p=nodo_anterior.copy()
            accion=lista_acciones.pop()
            if accion=="up":
                aux=lista_p[index]
                lista_p[index]=lista_p[index-3]
                lista_p[index-3]=aux
            if accion=="down":
                aux=lista_p[index]
                lista_p[index]=lista_p[index+3]
                lista_p[index+3]=aux
            if accion== "right":
                aux=lista_p[index]
                lista_p[index]=lista_p[index+1]
                lista_p[index+1]=aux
            if accion=="left":
                aux=lista_p[index]
                lista_p[index]=lista_p[index-1]
                lista_p[index-1]=aux

            lista_n_sucesores.append(lista_p)

        return lista_n_sucesores
    
    @staticmethod
    def peso_heuristica(tupla_de_lista_tuplas):
        return tupla_de_lista_tuplas[2]#+len(tupla_de_lista_tuplas[1])
    
    @staticmethod
    def get_min_heuristic(lista_nodos_posible):
        nodo_mejor_heuristica=min(lista_nodos_posible, key=AStar8Puzzle.peso_heuristica)
        return nodo_mejor_heuristica
    
    @staticmethod
    def formato_comp_tuplas(nodo_actual, path_p, goal_node, reached):
        acciones_posibles=AStar8Puzzle.get_n_sucesores(nodo_actual)
        lista_sucesores=AStar8Puzzle.generate_n_sucesores(acciones_posibles, nodo_actual)
        sucesores_tuplas=[]
        for node in lista_sucesores:
            node_tupla=tuple(node)

            if node_tupla not in reached:
                path_nuevo=path_p+[node]
                score_nueva= AStar8Puzzle.get_score(node,goal_node)
                sucesores_tuplas.append((node,path_nuevo,score_nueva))
                reached.add(node_tupla)

        return sucesores_tuplas
    
    @staticmethod
    def imprimir_matriz(array_m):
        print("Matriz :")
        for i in range(0, len(array_m), 3):
            print(' '.join(map(str, array_m[i:i+3])))
        ####print("Fin de Matriz")
        print('-' * 20)
              
    @staticmethod
    def solve(scramble, solution):
        start_time = time.time()
        
        full_path = []
        reached=set()
        fila_a_star=[(scramble,[scramble],AStar8Puzzle.get_score(scramble,solution))]
        i=0
        while fila_a_star:
            i=i+1
            zip_nodo=AStar8Puzzle.get_min_heuristic(fila_a_star)

            fila_a_star.remove(zip_nodo)
            nodo_p, path_p, score_p = zip_nodo
            full_path.append(nodo_p)

            if nodo_p==solution:
                end_time = time.time()
                elapsed_time = end_time - start_time
                return nodo_p, path_p,full_path, score_p, elapsed_time

            tuplas_sucesores=AStar8Puzzle.formato_comp_tuplas(nodo_p,path_p,solution,reached)
            fila_a_star= fila_a_star+tuplas_sucesores
    

# Implementation

In [114]:
# Initialize Pygame
pygame.init()

# Set the dimensions of the window
screen_width = 800
screen_height = 600

# Create the screen
screen = pygame.display.set_mode((screen_width, screen_height))

# Set the title of the window
pygame.display.set_caption("8-Puzzle")

# Create a clock object to control the frame rate
clock = pygame.time.Clock()

# Set the desired frames per second (fps)
fps = 30

# Create button instances
scramble_button = Button(25, 250, 150, 100, "Scramble", DARK_ORANGE, ORANGE)
a_star_button = Button(25, 100, 150, 100, "A*", DARK_BLUE, BLUE)
greedy_button = Button(25, 400, 150, 100, "Greedy", DARK_LIME, LIME)

# Initial states
scramble = [2, 8, 3, 1, ' ', 5, 4, 7, 6]
solution = [1, 2, 3, 4, 5, 6, 7, 8, ' ']
state = solution
button_state = "Scramble"  # Initial button state

# Main loop
running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if button_state == "Scramble" and scramble_button.is_clicked(event):
            state = scramble
            button_state = "Choose Algorithm"
        elif button_state == "Choose Algorithm":
            if a_star_button.is_clicked(event):
                nodo_p, path_p, full_path, score_p, elapsed_time = AStar8Puzzle.solve(scramble, solution)
                
                for n in full_path:
                    screen.fill(WHITE)
                    draw_contour(screen, 600, DARK_BLUE)
                    draw_contour(screen, 585, BLUE)
                    draw_contour(screen, 552, GREY)
                    draw_pieces(screen, n)
                    draw_text(screen, "Solving...", 25, 280, 50, BLUE)
                    pygame.display.flip()
                    pygame.time.delay(30)
                
                
                draw_text(screen, "TIME:", 25, 100, 50, ORANGE)
                draw_text(screen, str(round_better(elapsed_time,2)) + " s", 25, 135, 45, ORANGE)
                pygame.display.flip()
                pygame.time.delay(4000)

                for n in path_p:
                    screen.fill(WHITE)
                    draw_contour(screen, 600, DARK_LIME)
                    draw_contour(screen, 585, LIME)
                    draw_contour(screen, 552, GREY)
                    draw_pieces(screen, n)
                    draw_text(screen, "Solution", 25, 280, 50, LIME)
                    pygame.display.flip()
                    pygame.time.delay(200)
                pygame.time.delay(2000)
                state = solution
                button_state = "Scramble"
                scramble_button.set_text("Scramble")
                scramble_button.set_color(DARK_ORANGE)
            elif greedy_button.is_clicked(event):
                print("Greedy Algorithm (Not implemented)")  # Or any other indication for Greedy

    # Fill the screen with white color
    screen.fill(WHITE)

    # Draw the initial state of the puzzle
    draw_contour(screen, 600, DARK_BROWN)
    draw_contour(screen, 585, BROWN)
    draw_contour(screen, 552, GREY)
    draw_pieces(screen, state)

    if button_state == "Scramble":
        scramble_button.draw(screen)
    elif button_state == "Choose Algorithm":
        a_star_button.draw(screen)
        greedy_button.draw(screen)

    # Update the display
    pygame.display.flip()

    # Limit the frame rate
    clock.tick(fps)

# Quit Pygame
pygame.quit()

Greedy Algorithm (Not implemented)
Greedy Algorithm (Not implemented)
